### Initial EDA and Linear Regression Viability
- Data clean up, remove outliers
- Apply inflation to Gross and Budget

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import re
import cpi
from datetime import datetime

/opt/anaconda3/lib/python3.9/site-packages/cpi/__init__.py:46: StaleDataWarning: CPI data is out of date. To accurately inflate to today's dollars, you must run `cpi.update()`.
  warnings.warn(StaleDataWarning())


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#bring data together
batch1 = pd.read_csv('/Users/jennihawk/Desktop/Thriller Project/Movie_Linear_Regression/thriller_csv_files/thrillers_1_to_350_year_budget.csv')
batch2 = pd.read_csv('/Users/jennihawk/Desktop/Thriller Project/Movie_Linear_Regression/thriller_csv_files/thrillers_351_700rows.csv')
batch3 = pd.read_csv('/Users/jennihawk/Desktop/Thriller Project/Movie_Linear_Regression/thriller_csv_files/thrillers_751_1150rows.csv')

all_movies = [batch1, batch2, batch3]

movie_df = pd.concat(all_movies)

In [4]:
movie_df.reset_index(drop=True, inplace=True)

In [5]:
#movie_df

#### Replace current budget column with new, updated column values

In [6]:
all_budget = pd.read_csv('/Users/jennihawk/Desktop/Thriller Project/Movie_Linear_Regression/thriller_csv_files/all_data_budgetcol_only.csv')

In [7]:
all_budget.head()

,Title,Budget
0,The Dark Knight,185000000.0
1,The Hunger Games: Catching Fire,130000000.0
2,The Hunger Games,78000000.0
3,Jurassic Park,63000000.0
4,Furious 7,190000000.0


In [8]:
# bring in new budget column from all_budget df
movie_df = pd.merge(movie_df, all_budget, on="Title")
movie_df.head()

,Title,Duration,Genre,Rating,Gross,Director,Lead,Budget_x,Year,Budget_y
0,The Dark Knight,152 min,"Action, Crime, Drama",PG-13,534.86,Christopher Nolan,Christian Bale,185000000.0,2008,185000000.0
1,The Hunger Games: Catching Fire,146 min,"Action, Adventure, Drama",PG-13,424.67,Francis Lawrence,Jennifer Lawrence,130000000.0,2013,130000000.0
2,The Hunger Games,142 min,"Action, Adventure, Sci-Fi",PG-13,408.01,Gary Ross,Jennifer Lawrence,78000000.0,2012,78000000.0
3,Jurassic Park,127 min,"Action, Adventure, Sci-Fi",PG-13,402.45,Steven Spielberg,Sam Neill,63000000.0,1993,63000000.0
4,Furious 7,137 min,"Action, Crime, Thriller",PG-13,353.01,James Wan,Vin Diesel,190000000.0,2015,190000000.0


In [9]:
#remove old budget column
movie_df.drop(['Budget_x'], axis=1, inplace=True)
movie_df.head()

,Title,Duration,Genre,Rating,Gross,Director,Lead,Year,Budget_y
0,The Dark Knight,152 min,"Action, Crime, Drama",PG-13,534.86,Christopher Nolan,Christian Bale,2008,185000000.0
1,The Hunger Games: Catching Fire,146 min,"Action, Adventure, Drama",PG-13,424.67,Francis Lawrence,Jennifer Lawrence,2013,130000000.0
2,The Hunger Games,142 min,"Action, Adventure, Sci-Fi",PG-13,408.01,Gary Ross,Jennifer Lawrence,2012,78000000.0
3,Jurassic Park,127 min,"Action, Adventure, Sci-Fi",PG-13,402.45,Steven Spielberg,Sam Neill,1993,63000000.0
4,Furious 7,137 min,"Action, Crime, Thriller",PG-13,353.01,James Wan,Vin Diesel,2015,190000000.0


In [10]:
#rename 'budget_y' to 'budget'
movie_df.rename(columns = {'Budget_y':'Budget_Update'}, inplace = True)
movie_df.head()

,Title,Duration,Genre,Rating,Gross,Director,Lead,Year,Budget_Update
0,The Dark Knight,152 min,"Action, Crime, Drama",PG-13,534.86,Christopher Nolan,Christian Bale,2008,185000000.0
1,The Hunger Games: Catching Fire,146 min,"Action, Adventure, Drama",PG-13,424.67,Francis Lawrence,Jennifer Lawrence,2013,130000000.0
2,The Hunger Games,142 min,"Action, Adventure, Sci-Fi",PG-13,408.01,Gary Ross,Jennifer Lawrence,2012,78000000.0
3,Jurassic Park,127 min,"Action, Adventure, Sci-Fi",PG-13,402.45,Steven Spielberg,Sam Neill,1993,63000000.0
4,Furious 7,137 min,"Action, Crime, Thriller",PG-13,353.01,James Wan,Vin Diesel,2015,190000000.0


In [11]:
movie_df.columns

Index(['Title', 'Duration', 'Genre', 'Rating', 'Gross', 'Director', 'Lead',
       'Year', 'Budget_Update'],
      dtype='object')

In [12]:
#27 missing budget values
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1121 entries, 0 to 1120
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Title          1121 non-null   object 
 1   Duration       1121 non-null   object 
 2   Genre          1121 non-null   object 
 3   Rating         1121 non-null   object 
 4   Gross          1121 non-null   float64
 5   Director       1121 non-null   object 
 6   Lead           1121 non-null   object 
 7   Year           1121 non-null   object 
 8   Budget_Update  1094 non-null   float64
dtypes: float64(2), object(7)
memory usage: 87.6+ KB


In [13]:
# shows rows that have null values
movie_df[movie_df.isnull().any(axis=1)]

,Title,Duration,Genre,Rating,Gross,Director,Lead,Year,Budget_Update
73,A Quiet Place Part II,97 min,"Drama, Horror, Sci-Fi",PG-13,160.07,John Krasinski,Emily Blunt,2020,NaN
421,Temptation: Confessions of a Marriage Counselor,111 min,"Drama, Romance, Thriller",PG-13,51.98,Tyler Perry,Jurnee Smollett,2013,NaN
435,Jai Lava Kusa,150 min,"Action, Drama, Thriller",Not Rated,51.00,K.S. Ravindra,N.T. Rama Rao Jr.,2017,NaN
665,Sharky's Machine,122 min,"Action, Crime, Drama",R,35.61,Burt Reynolds,Burt Reynolds,1981,NaN
728,Blade Runner,117 min,"Action, Drama, Sci-Fi",R,32.87,Ridley Scott,Harrison Ford,1982,NaN
757,9,79 min,"Animation, Action, Adventure",PG-13,31.74,Shane Acker,Elijah Wood,I 2009,NaN
809,Deceived,104 min,"Crime, Drama, Mystery",PG-13,28.74,Damian Harris,Goldie Hawn,1991,NaN
823,That Darn Cat!,116 min,"Comedy, Crime, Family",Approved,28.06,Robert Stevenson,Hayley Mills,1965,NaN
865,Æon Flux,93 min,"Action, Adventure, Sci-Fi",PG-13,25.86,Karyn Kusama,Charlize Theron,2005,NaN
887,The VVitch: A New-England Folktale,92 min,"Drama, Fantasy, History",R,25.14,Robert Eggers,Anya Taylor-Joy,2015,NaN


In [14]:
#sum of rows with at least one missing data
total_miss = movie_df.isnull().any(axis=1).sum()
total_miss

27

In [15]:
#drop the 27 nulls in budget column
movie_df.dropna(subset=['Budget_Update'], inplace=True)

In [16]:
# drop outlier The Curse of Frankenstein 1957
movie_df.drop(index = 1084, inplace = True)

In [17]:
#find original Jaws in 'Title' use 'contains' to find substring in column
find_jaws = movie_df.loc[movie_df['Title'].str.contains("Jaws", case=False)]
find_jaws

,Title,Duration,Genre,Rating,Gross,Director,Lead,Year,Budget_Update
14,Jaws,124 min,"Adventure, Thriller",PG,260.75,Steven Spielberg,Roy Scheider,1975,7000000.0
166,Jaws 2,116 min,"Adventure, Horror, Thriller",PG,102.92,Jeannot Szwarc,Roy Scheider,1978,20000000.0
503,Jaws 3-D,99 min,"Adventure, Horror, Thriller",PG,45.52,Joe Alves,Dennis Quaid,1983,20500000.0
994,Jaws: The Revenge,89 min,"Adventure, Horror, Thriller",PG-13,20.76,Joseph Sargent,Lorraine Gary,1987,23000000.0


In [18]:
# drop outlier Jaws 1975 - Jaws is a complete unique phenomenon that would distort charts
movie_df.drop(index = 14, inplace = True)

#### Clean up

In [19]:
#remove any leading/trailing in column titles
movie_df.columns = movie_df.columns.map(str.strip)
#movie_df

In [20]:
# Duration cleanup + change to float
movie_df['Duration'] = movie_df['Duration'].str.replace('min', '').astype(float)

In [21]:
movie_df['Budget_Update'] = movie_df['Budget_Update'].astype(float)

In [22]:
pd.set_option('display.max_rows', None)

In [23]:
# Cast year as string to remove extra chars in front of some columns year
movie_df['Year'] = movie_df['Year'].astype(str)

In [24]:
movie_df['Duration'] = movie_df['Duration'].astype(int)

In [25]:
movie_df.head()

,Title,Duration,Genre,Rating,Gross,Director,Lead,Year,Budget_Update
0,The Dark Knight,152,"Action, Crime, Drama",PG-13,534.86,Christopher Nolan,Christian Bale,2008,185000000.0
1,The Hunger Games: Catching Fire,146,"Action, Adventure, Drama",PG-13,424.67,Francis Lawrence,Jennifer Lawrence,2013,130000000.0
2,The Hunger Games,142,"Action, Adventure, Sci-Fi",PG-13,408.01,Gary Ross,Jennifer Lawrence,2012,78000000.0
3,Jurassic Park,127,"Action, Adventure, Sci-Fi",PG-13,402.45,Steven Spielberg,Sam Neill,1993,63000000.0
4,Furious 7,137,"Action, Crime, Thriller",PG-13,353.01,James Wan,Vin Diesel,2015,190000000.0


In [26]:
#make 'Gross' same distribution as 'Budget'
movie_df['Gross'] = movie_df['Gross'].apply(lambda x: x * 1000000)

In [27]:
movie_df.head()

,Title,Duration,Genre,Rating,Gross,Director,Lead,Year,Budget_Update
0,The Dark Knight,152,"Action, Crime, Drama",PG-13,534860000.0,Christopher Nolan,Christian Bale,2008,185000000.0
1,The Hunger Games: Catching Fire,146,"Action, Adventure, Drama",PG-13,424670000.0,Francis Lawrence,Jennifer Lawrence,2013,130000000.0
2,The Hunger Games,142,"Action, Adventure, Sci-Fi",PG-13,408010000.0,Gary Ross,Jennifer Lawrence,2012,78000000.0
3,Jurassic Park,127,"Action, Adventure, Sci-Fi",PG-13,402450000.0,Steven Spielberg,Sam Neill,1993,63000000.0
4,Furious 7,137,"Action, Crime, Thriller",PG-13,353010000.0,James Wan,Vin Diesel,2015,190000000.0


#### Clean up extra chars in year 

In [28]:
# remove extra chars in year column
def my_function(x):
    return x[-4:]
#saves over the column
movie_df['Year'] = movie_df['Year'].map(my_function)       

#### Inflation work
https://pypi.org/project/cpi/

In [29]:
movie_df['Year']=pd.to_datetime(movie_df['Year'], format='%Y')

In [30]:
movie_df['Year'] = movie_df['Year'].dt.year

In [32]:
movie_df['Budget_Adjust'] = movie_df.apply(lambda x: cpi.inflate(x.Budget_Update, x.Year), axis=1)

In [33]:
movie_df['Budget_Adjust'] = movie_df['Budget_Adjust'].round(decimals = 1)

In [34]:
movie_df['Gross_Adjust'] = movie_df.apply(lambda x: cpi.inflate(x.Gross, x.Year), axis=1)

In [35]:
movie_df['Gross_Adjust'] = movie_df['Gross_Adjust'].round(decimals = 1)

In [36]:
movie_df.head(3)

,Title,Duration,Genre,Rating,Gross,Director,Lead,Year,Budget_Update,Budget_Adjust,Gross_Adjust
0,The Dark Knight,152,"Action, Crime, Drama",PG-13,534860000.0,Christopher Nolan,Christian Bale,2008,185000000.0,232832101.7,673149069.9
1,The Hunger Games: Catching Fire,146,"Action, Adventure, Drama",PG-13,424670000.0,Francis Lawrence,Jennifer Lawrence,2013,130000000.0,151212884.8,493965967.5
2,The Hunger Games,142,"Action, Adventure, Sci-Fi",PG-13,408010000.0,Gary Ross,Jennifer Lawrence,2012,78000000.0,92056674.0,481539019.7


In [38]:
#spot check 
# test = cpi.inflate(250000000, 2017, to=2021)
# test

#### Drop ratings except PG-13, PG, R

In [40]:
movie_df.Rating.value_counts()

R            630
PG-13        383
PG            72
Not Rated      2
Approved       2
G              1
M/PG           1
X              1
Name: Rating, dtype: int64

In [50]:
movie_df.drop(movie_df.index[movie_df['Rating'] == 'Not Rated'], inplace=True)

In [52]:
movie_df.drop(movie_df.index[movie_df['Rating'] == 'Approved'], inplace=True)
movie_df.drop(movie_df.index[movie_df['Rating'] == 'G'], inplace=True)
movie_df.drop(movie_df.index[movie_df['Rating'] == 'M/PG'], inplace=True)
movie_df.drop(movie_df.index[movie_df['Rating'] == 'X'], inplace=True)

In [53]:
movie_df.Rating.value_counts()

R        630
PG-13    383
PG        72
Name: Rating, dtype: int64

In [54]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1085 entries, 0 to 1119
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Title          1085 non-null   object 
 1   Duration       1085 non-null   int64  
 2   Genre          1085 non-null   object 
 3   Rating         1085 non-null   object 
 4   Gross          1085 non-null   float64
 5   Director       1085 non-null   object 
 6   Lead           1085 non-null   object 
 7   Year           1085 non-null   int64  
 8   Budget_Update  1085 non-null   float64
 9   Budget_Adjust  1085 non-null   float64
 10  Gross_Adjust   1085 non-null   float64
dtypes: float64(4), int64(2), object(5)
memory usage: 101.7+ KB


In [55]:
#movie_df.to_csv('movies_cleaned.csv', index=False)